In [3]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
  temperature=0.1
)

In [4]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content = "You are a geography expert. And you only replay in Korean."),
  AIMessage(content="안녕하세요, 제 이름은 형진입니다."),
  HumanMessage(content="What is the distance between Mexico and Thailand. Also, What is your name?")
]

chat.predict_messages(messages)

AIMessage(content='안녕하세요! 저의 이름은 형진입니다. 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.')